# Exercice : Langage des signes

## Création du learning et validation dataset

In [1]:
import numpy as np
import pandas as pd
from utils.data_processing import load_dataframe_from_file, store_data

def flatten_row(row):
    input_flat = row['inputs'].flatten().tolist()
    label_flat = row['label'].tolist()
    return input_flat + label_flat

# DataFrame de base
df = load_dataframe_from_file('../datasets/LangageDesSignes/data_formatted.csv', 5)

# Creation du learning dataset
df['class'] = df['label'].apply(lambda x: int(np.argmax(x)))

df_learning = (
    df.groupby('class')
      .apply(lambda x: x.sample(n=50, random_state=42))
      .reset_index(drop=True)
)

df_learning = df_learning.drop(columns=['class'])
df_learning['inputs'] = df_learning['inputs'].apply(lambda x: x[1:] if isinstance(x, np.ndarray) else x)
print(df_learning)

df_learning_good = pd.DataFrame(df_learning.apply(flatten_row, axis=1).tolist())
df_learning_good.to_csv('../datasets/LangageDesSignes/learning_dataset.csv', index=False, sep=",", header=False)


# Creation du validation dataset sans la colonne label
df['class'] = df['label'].apply(lambda x: int(np.argmax(x)))

selected_indices = df.groupby('class').apply(lambda x: x.sample(n=50, random_state=42)).index.get_level_values(1)

df_remaining = df.drop(index=selected_indices)

df_validation = df_remaining[['inputs']].reset_index(drop=True)
df_validation['inputs'] = df_validation['inputs'].apply(lambda x: x[1:] if isinstance(x, np.ndarray) else x)
print(df_validation)

df_validation_good = pd.DataFrame(df_validation['inputs'].apply(lambda x: x.flatten().tolist()).tolist())
df_validation_good.to_csv('../datasets/LangageDesSignes/validation_dataset.csv', index=False, sep=",", header=False)

# test_learn = load_dataframe_from_file("../datasets/LangageDesSignes/learning_dataset.csv",5)
# test_valid = load_dataframe_from_file("../datasets/LangageDesSignes/validation_dataset.csv",0)
# print(test_learn)
# print(test_valid)

                                               inputs
0   [1.0, 0.0, 0.0, -0.296, 0.1203703703703703, -0...
1   [1.0, 0.0, 0.0, -0.2824074074074074, 0.1172839...
2   [1.0, 0.0, 0.0, -0.336734693877551, 0.13580246...
3   [1.0, 0.0, 0.0, -0.3271889400921659, 0.1085714...
4   [1.0, 0.0, 0.0, -0.3302752293577982, 0.1695906...
5   [1.0, 0.0, 0.0, -0.2785388127853881, 0.1121951...
6   [1.0, 0.0, 0.0, -0.3, 0.0914634146341463, -0.5...
7   [1.0, 0.0, 0.0, -0.3050314465408805, 0.0876494...
8   [1.0, 0.0, 0.0, -0.3121693121693121, 0.15, -0....
9   [1.0, 0.0, 0.0, -0.300395256916996, 0.04975124...
10  [1.0, 0.0, 0.0, -0.5233644859813084, 0.0646153...
11  [1.0, 0.0, 0.0, -0.6417910447761194, 0.0899653...
12  [1.0, 0.0, 0.0, -0.4554455445544554, 0.0449438...
13  [1.0, 0.0, 0.0, -0.5454545454545454, 0.0629370...
14  [1.0, 0.0, 0.0, -0.6440677966101694, 0.0916334...
15  [1.0, 0.0, 0.0, -0.6538461538461539, 0.125, -1...
16  [1.0, 0.0, 0.0, -0.6379310344827587, 0.0912698...
17  [1.0, 0.0, 0.0, -0.63076

C:\Users\Axel\AppData\Local\Temp\ipykernel_23640\102884436.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=50, random_state=42))
C:\Users\Axel\AppData\Local\Temp\ipykernel_23640\102884436.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  selected_indices = df.groupby('class').apply(lambda x: x.sample(n=50, random_state=42)).index.get_level_values(1)


## Apprentissage du réseau

In [12]:
import numpy as np
from multilayer.layer import Layer
from multilayer.model import Model
from utils.data_processing import load_dataframe_from_file

model = Model([
    Layer(units=10, activation="sigmoid", input_size=42),
    Layer(units=5, activation="sigmoid"),
])

training_df = load_dataframe_from_file("../datasets/LangageDesSignes/learning_dataset.csv", 5)
x_train = np.array([x[1:] for x in training_df["inputs"]], dtype=float)
y_train = np.stack(training_df["label"].values)

history = model.fit(x_train, y_train, 0.5, 0.001, 2000)

ValueError: operands could not be broadcast together with shapes (1250,) (250,5) 